In [5]:
import keras
import time
import numpy as np
import tensorflow as tf
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from keras.applications.resnet import ResNet50
from sklearn.model_selection import train_test_split

In [6]:
# Loading data
data = np.load('/content/drive/MyDrive/data_224.npy')
labels = np.load('/content/drive/MyDrive/labels_224.npy')

# One hot encode labels
unique, inverse = np.unique(labels, return_inverse=True)
labels = np.eye(unique.shape[0])[inverse]

# Loading data2
data2 = np.load('/content/drive/MyDrive/data2_224.npy')
labels2 = np.load('/content/drive/MyDrive/labels2_224.npy')

# One hot encode labels
unique, inverse = np.unique(labels2, return_inverse=True)
labels2 = np.eye(unique.shape[0])[inverse]

data = np.concatenate((data, data2))
labels = np.concatenate((labels, labels2))

# Reshape image array
data = np.reshape(data,(len(data),224,224,3))

print("Data   : ",data.shape)
print("Labels : ",labels.shape)

Data   :  (7890, 224, 224, 3)
Labels :  (7890, 2)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.30)

print("Training Data   : ",X_train.shape)
print("Training Labels : ",y_train.shape)
print("Testing  Data   : ",X_test.shape)
print("Testing  Labels : ",y_test.shape)

Training Data   :  (5523, 224, 224, 3)
Training Labels :  (5523, 2)
Testing  Data   :  (2367, 224, 224, 3)
Testing  Labels :  (2367, 2)


In [9]:
resnet_model = ResNet50(weights='imagenet',include_top=False)

# Pooling/ flat
pooling = keras.layers.GlobalAveragePooling2D()(resnet_model.output)

# Adding fully connected layer 1
pred = keras.layers.Dense(2, activation='softmax', name="pred")(pooling)

# Final model
model = keras.models.Model(inputs = resnet_model.input, outputs=pred)

# Options setting
options = keras.optimizers.SGD(learning_rate=1e-3,decay=1e-6,momentum=0.9)

model.compile(optimizer=options, metrics=["accuracy"],loss='categorical_crossentropy')

# saving the best model 
filepath="/content/drive/MyDrive/mask-detection.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# train the model
start = time.time()
model.fit(X_train, y_train, batch_size = 64, epochs = 5, shuffle=True, callbacks = callbacks_list, validation_data=(X_test, y_test))
end = time.time()

# testing the model
start2 = time.time()
_,train_acc = model.evaluate(X_train,y_train, batch_size=1)
_,test_acc = model.evaluate(X_test,y_test, batch_size=1)
end2 = time.time()

print("======================================================================================================================================")
print("Training time : ",end-start)
print("Testing time  : ",end2-start2)
print("Training Accuracy : ",train_acc*100)
print("Testing  Accuracy : ",test_acc*100)
print("======================================================================================================================================")

94781440/94765736 [==============================] - 1s 0us/step
Epoch 1/5
87/87 [==============================] - 195s 2s/step - loss: 0.4158 - accuracy: 0.7961 - val_loss: 0.0301 - val_accuracy: 0.9899

Epoch 00001: val_accuracy improved from -inf to 0.98986, saving model to /content/drive/MyDrive/mask-detection.hdf5


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/5
87/87 [==============================] - 123s 1s/step - loss: 0.0217 - accuracy: 0.9965 - val_loss: 0.0233 - val_accuracy: 0.9937

Epoch 00002: val_accuracy improved from 0.98986 to 0.99366, saving model to /content/drive/MyDrive/mask-detection.hdf5
Epoch 3/5
87/87 [==============================] - 123s 1s/step - loss: 0.0121 - accuracy: 0.9973 - val_loss: 0.0182 - val_accuracy: 0.9958

Epoch 00003: val_accuracy improved from 0.99366 to 0.99578, saving model to /content/drive/MyDrive/mask-detection.hdf5
Epoch 4/5
87/87 [==============================] - 123s 1s/step - loss: 0.0074 - accuracy: 0.9998 - val_loss: 0.0158 - val_accuracy: 0.9962

Epoch 00004: val_accuracy improved from 0.99578 to 0.99620, saving model to /content/drive/MyDrive/mask-detection.hdf5
Epoch 5/5
87/87 [==============================] - 123s 1s/step - loss: 0.0043 - accuracy: 0.9996 - val_loss: 0.0156 - val_accuracy: 0.9945

Epoch 00005: val_accuracy did not improve from 0.99620
2367/2367 [=============

In [16]:
from keras.models import load_model
model=load_model("/content/drive/MyDrive/mask-detection.hdf5")
_,train_acc = model.evaluate(X_train,y_train, batch_size=1)
_,test_acc = model.evaluate(X_test,y_test, batch_size=1)

2367/2367 [==============================] - 55s 23ms/step - loss: 0.0158 - accuracy: 0.9962
